# Understand how to work with RAGAS

In [1]:
import ragas

## Sample Code: Very structured!

In [2]:
from ragas.prompt import PydanticPrompt
from pydantic import BaseModel, Field

class MyInput(BaseModel):
    question: str = Field(description="The question to answer")

class MyOutput(BaseModel):
    answer: str = Field(description="The answer to the question")

class MyPrompt(PydanticPrompt[MyInput,MyOutput]):
    instruction = "Answer the given question"
    input_model = MyInput
    output_model = MyOutput
    examples = [
        (
            MyInput(question="Who's building the opensource standard for LLM app evals?"),
            MyOutput(answer="Ragas")
        )
    ]

## 1. Evaluation with non-LLM metric: Doesnt work too well.

In [ ]:
from ragas import SingleTurnSample
from ragas.metrics import BleuScore

test_data = {
    "user_input": "summarise given text\nThe company reported an 8% rise in Q3 2024, driven by strong performance in the Asian market. Sales in this region have significantly contributed to the overall growth. Analysts attribute this success to strategic marketing and product localization. The positive trend in the Asian market is expected to continue into the next quarter.",
    "response": "The company experienced an 8% increase in Q3 2024, largely due to effective marketing strategies and product adaptation, with expectations of continued growth in the coming quarter.",
    "reference": "The company reported an 8% growth in Q3 2024, primarily driven by strong sales in the Asian market, attributed to strategic marketing and localized products, with continued growth anticipated in the next quarter."
}
metric = BleuScore()
test_data = SingleTurnSample(**test_data)
metric.single_turn_score(test_data)

0.13718598426177148

Better if one uses LLM Metrics. 

In [4]:
from dotenv import load_dotenv
import os
class OPENAIKeyNotFoundError(Exception):
    pass
load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    raise OPENAIKeyNotFoundError("OpenAI API Key not found in environment variables")
if "SOURCE_DOCS_DIR" not in os.environ:
    raise ValueError("SOURCE_DOCS_DIR not found in environment variables")

Wrap LLM in LangchainLLMWrapper

In [5]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

2. Evaluation with Aspect Critic (LLM metric)

In [6]:
from ragas import SingleTurnSample
from ragas.metrics import AspectCritic

test_data = {
    "user_input": "summarise given text\nThe company reported an 8% rise in Q3 2024, driven by strong performance in the Asian market. Sales in this region have significantly contributed to the overall growth. Analysts attribute this success to strategic marketing and product localization. The positive trend in the Asian market is expected to continue into the next quarter.",
    "response": "The company experienced an 8% increase in Q3 2024, largely due to effective marketing strategies and product adaptation, with expectations of continued growth in the coming quarter.",
}

metric = AspectCritic(name="summary_accuracy",llm=evaluator_llm, definition="Verify if the summary is accurate.")
test_data = SingleTurnSample(**test_data)
await metric.single_turn_ascore(test_data)

1

## Applying Retrieval to Our Use Case: Question 3  

### Question:  
**"Zitiere die genauen Anforderungen für die Bewerbung nach der Bewertungsmatrix?"**  

While this question is highly specific, it does not explicitly request the field **"Kriteriengewichtung"**.  

### Observations:  
- Each requirement has a corresponding **criteria weight** assigned to it.  
- Our agent has been able to **retrieve the requirements accurately (for the most part)**.  
- However, the **attribution of requirements to their corresponding criteria weightings was missing**.  
- When extending the query with the phrase **"include the corresponding criteria weighting"**, the agent **assigned incorrect values**.  

### Hypothesis:  
The incorrect attribution of weighting values is likely due to the **tabular structure** of the PDF file.  
- The **_unstructured_ PDF loader** we are currently using **may not be suitable** for handling tabular data.  
- To improve retrieval accuracy, we will experiment with **alternative PDF loaders** and evaluate their performance in extracting both requirements and their corresponding criteria weightings correctly.  


In [7]:
from langchain_community.document_loaders import TextLoader, UnstructuredPDFLoader
from pathlib import Path
source_path = Path.cwd().parent / Path(os.getenv("SOURCE_DOCS_DIR")) / "A_Bewertungsmatrix.pdf" #type: ignore
loader = UnstructuredPDFLoader(source_path)
docs = loader.load()


In [8]:
from pprint import pprint
pprint(vars(docs[0]))
document = docs.pop()

{'id': None,
 'metadata': {'source': '/home/swanckel/projects/pdfchat_rag/source_docs/A_Bewertungsmatrix.pdf'},
 'page_content': '301-23-45ind-30052\n'
                 '\n'
                 'REZ NRW 45ind JC Dortmund\n'
                 '\n'
                 'Bewertungsmatrix\n'
                 '\n'
                 'Nr.\n'
                 '\n'
                 'Antwort\n'
                 '\n'
                 'I\n'
                 '\n'
                 'I.1\n'
                 '\n'
                 'Auftragsbezogene Zusammenarbeit auf dem regionalen '
                 'Ausbildungs- und Arbeitsmarkt Auftragsbezogene '
                 'Zusammenarbeit Benennen Sie die für die Maßnahmedurchführung '
                 're-levanten regionalen Akteure (Bezeichnung und Ort) und '
                 'beschreiben Sie Art und Umfang der Zusammenarbeit ab '
                 'Maßnahmebeginn unter Berücksichtigung der Zielsetzung und '
                 'Zielgruppe der konkreten Maßnahme.\n'
    

# Test that Ragas Works as expected. 
### Lets construct ground truths and expect an accuracy score of 1

In [9]:
ORIGINAL_QUESTION = "Zitiere die genauen Anforderungen für die Bewerbung nach der Bewertungsmatrix?"
template = """
You are a chatbot that helps with answering questions. 
You shall answer the questions only on the context provided.
Avoid providing answers that are not relevant to the context.
Avoid providing answers that are not relevant to the question.
Answer the question in the language of the question.

Answer given the following context:
{context}

Question: {question}
"""

In [10]:
from pydantic import BaseModel, Field
class TestSample(BaseModel):
    user_input: str = Field(description="Question, context and possible template")
    response: str = Field(description="The answer to the question")


In [11]:
_dataset = []
_dataset.append(TestSample(user_input=template.format(context=document.page_content, question=ORIGINAL_QUESTION), response=
"""
Die genauen Anforderungen für die Bewerbung nach der Bewertungsmatrix sind:

I. Auftragsbezogene Zusammenarbeit auf dem regionalen Ausbildungs- und Arbeitsmarkt
I.1 Benennen Sie die für die Maßnahmedurchführung relevanten regionalen Akteure (Bezeichnung und Ort) und beschreiben Sie Art und Umfang der Zusammenarbeit ab Maßnahmebeginn unter Berücksichtigung der Zielsetzung und Zielgruppe der konkreten Maßnahme.

II. Teilnahmebezogene Vorgehensweise
II.1 Anbahnungsphase
Beschreiben Sie anhand eines konkreten Beispiels, wie das Halten bzw. Wiederherstellen des Kontaktes und das Schaffen einer Vertrauensbasis zu einer Person, die den Kontakt zur Integrationsfachkraft abgebrochen hat und nicht auf Einladungen reagiert, erfolgen soll. Gehen Sie dabei auch auf mögliche innovative Ansätze zur Kontaktaufnahme ein.
II.2 Akutphase
Beschreiben Sie beispielhaft die Akutphase für eine teilnehmende Person mit einer akuten Drogenproblematik bei der zudem finanzielle Rückstände bei der Krankenkasse bestehen.
II.3 Coachingphase
Zu beschreiben ist die strategische und methodische Vorgehensweise zur Aktivierung der Teilnehmenden und Heranführung an den Arbeitsmarkt. Stellen Sie exemplarisch den Verlauf der Coachingphase für eine teilnehmende Person dieser Maßnahme dar. Berücksichtigen Sie bei Ihrer Darstellung die Inhalte unter B.2.3.

III. Personal
III.1 Personaleinsatz
Stellen Sie ihr Qualitätsmanagement bezogen auf das eingesetzte Personal dar. Führen Sie aus, wie Sie sicherstellen, dass das Personal die Maßnahme nach verbindlichen Qualitätskriterien durchführt. Erläutern Sie dabei das Zusammenwirken des in der Maßnahme eingesetzten Personals. Schildern Sie, wie Sie mit dem eingesetzten Personal die Zielsetzung der Maßnahme gemäß B.1.1 sowie die vorgegebene Kontaktdichte realisieren wollen. Berücksichtigen Sie dabei Ihre Gesamtstrategie zur Leistungserbringung sowie die Zusammenarbeit mit dem Bedarfsträger.

IV. Organisation
IV.1 Exemplarischer Maßnahmeverlauf
Stellen Sie exemplarisch den Verlauf der Maßnahme für eine teilnehmende Person einer Maßnahme lt. Leistungsverzeichnis/Losblatt mit, -anhaltende psychische Auffälligkeiten -Alkoholkonsum -Schuldenproblematik dar, der ein Wohnungsverlust wegen Mietschulden droht. Erläutern und begründen Sie das von Ihnen gewählte methodische Vorgehen."""
).model_dump())

In [12]:
TestSample(user_input=template.format(context=document.page_content, question=ORIGINAL_QUESTION), response=
"""
Die genauen Anforderungen für die Bewerbung nach der Bewertungsmatrix sind:

I. Auftragsbezogene Zusammenarbeit auf dem regionalen Ausbildungs- und Arbeitsmarkt
I.1 Benennen Sie die für die Maßnahmedurchführung relevanten regionalen Akteure (Bezeichnung und Ort) und beschreiben Sie Art und Umfang der Zusammenarbeit ab Maßnahmebeginn unter Berücksichtigung der Zielsetzung und Zielgruppe der konkreten Maßnahme.

II. Teilnahmebezogene Vorgehensweise
II.1 Anbahnungsphase
Beschreiben Sie anhand eines konkreten Beispiels, wie das Halten bzw. Wiederherstellen des Kontaktes und das Schaffen einer Vertrauensbasis zu einer Person, die den Kontakt zur Integrationsfachkraft abgebrochen hat und nicht auf Einladungen reagiert, erfolgen soll. Gehen Sie dabei auch auf mögliche innovative Ansätze zur Kontaktaufnahme ein.
II.2 Akutphase
Beschreiben Sie beispielhaft die Akutphase für eine teilnehmende Person mit einer akuten Drogenproblematik bei der zudem finanzielle Rückstände bei der Krankenkasse bestehen.
II.3 Coachingphase
Zu beschreiben ist die strategische und methodische Vorgehensweise zur Aktivierung der Teilnehmenden und Heranführung an den Arbeitsmarkt. Stellen Sie exemplarisch den Verlauf der Coachingphase für eine teilnehmende Person dieser Maßnahme dar. Berücksichtigen Sie bei Ihrer Darstellung die Inhalte unter B.2.3.

III. Personal
III.1 Personaleinsatz
Stellen Sie ihr Qualitätsmanagement bezogen auf das eingesetzte Personal dar. Führen Sie aus, wie Sie sicherstellen, dass das Personal die Maßnahme nach verbindlichen Qualitätskriterien durchführt. Erläutern Sie dabei das Zusammenwirken des in der Maßnahme eingesetzten Personals. Schildern Sie, wie Sie mit dem eingesetzten Personal die Zielsetzung der Maßnahme gemäß B.1.1 sowie die vorgegebene Kontaktdichte realisieren wollen. Berücksichtigen Sie dabei Ihre Gesamtstrategie zur Leistungserbringung sowie die Zusammenarbeit mit dem Bedarfsträger.

IV. Organisation
IV.1 Exemplarischer Maßnahmeverlauf
Stellen Sie exemplarisch den Verlauf der Maßnahme für eine teilnehmende Person einer Maßnahme lt. Leistungsverzeichnis/Losblatt mit, -anhaltende psychische Auffälligkeiten -Alkoholkonsum -Schuldenproblematik dar, der ein Wohnungsverlust wegen Mietschulden droht. Erläutern und begründen Sie das von Ihnen gewählte methodische Vorgehen."""
).model_dump()

{'user_input': '\nYou are a chatbot that helps with answering questions. \nYou shall answer the questions only on the context provided.\nAvoid providing answers that are not relevant to the context.\nAvoid providing answers that are not relevant to the question.\nAnswer the question in the language of the question.\n\nAnswer given the following context:\n301-23-45ind-30052\n\nREZ NRW 45ind JC Dortmund\n\nBewertungsmatrix\n\nNr.\n\nAntwort\n\nI\n\nI.1\n\nAuftragsbezogene Zusammenarbeit auf dem regionalen Ausbildungs- und Arbeitsmarkt Auftragsbezogene Zusammenarbeit Benennen Sie die für die Maßnahmedurchführung re-levanten regionalen Akteure (Bezeichnung und Ort) und beschreiben Sie Art und Umfang der Zusammenarbeit ab Maßnahmebeginn unter Berücksichtigung der Zielsetzung und Zielgruppe der konkreten Maßnahme.\n\nII\n\nTeilnahmebezogene Vorgehensweise\n\nII.1\n\nAnbahnungsphase Beschreiben Sie anhand eines konkreten Beispiels, wie das Halten bzw. Wiederherzustellen des Kontaktes und das 

In [13]:
from ragas import EvaluationDataset
eval_dataset = EvaluationDataset.from_list(_dataset)

In [14]:
print("Features in dataset:", eval_dataset.features())
print("Total samples in dataset:", len(eval_dataset))



Features in dataset: ['user_input', 'response']
Total samples in dataset: 1


In [15]:
from ragas import evaluate
results = evaluate(eval_dataset, metrics=[metric])
results

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

{'summary_accuracy': 1.0000}

In [16]:
weightings= {"Auftragsbezogene Zusammenarbeit auf dem regionalen Ausbildungs- und Arbeitsmarkt": "10",
               "Auftragsbezogene Zusammenarbeit": "2",
               "Teilnahmebezogene Vorgehensweise":"45",
               "Anbahnungsphase":"3",
               "Akutphase":"3",
               "Coachingphase":"3",
               "Personal":"15",
               "Personaleinsatz": "3",
               "Organisation":"30",
               "Exemplarischer Maßnahmeverlauf":"4",
}
weighting_factor_template = "Nenne mir die Kriteriengewichtung für {requirement}"
weight_factor_response = "Die Kriteriengewichtung für {requirement} beträgt {weighting_factor} GP."
for key,value in weightings.items():
    
    _dataset.append(TestSample(
        user_input=template.format(
            context=document.page_content, 
            question=weighting_factor_template.format(
                requirement=key)), 
        response=weight_factor_response.format(
            requirement=key, 
            weighting_factor=value))
    .model_dump()
    )

In [17]:
eval_dataset = EvaluationDataset.from_list(_dataset)
print("Features in dataset:", eval_dataset.features())
print("Total samples in dataset:", len(eval_dataset))

Features in dataset: ['user_input', 'response']
Total samples in dataset: 11


In [18]:
from ragas import evaluate
results = evaluate(eval_dataset, metrics=[metric])
results

Evaluating:   0%|          | 0/11 [00:00<?, ?it/s]

{'summary_accuracy': 1.0000}

In [19]:
results.to_pandas()


,user_input,response,summary_accuracy
0,\nYou are a chatbot that helps with answering ...,\nDie genauen Anforderungen für die Bewerbung ...,1
1,\nYou are a chatbot that helps with answering ...,Die Kriteriengewichtung für Auftragsbezogene Z...,1
2,\nYou are a chatbot that helps with answering ...,Die Kriteriengewichtung für Auftragsbezogene Z...,1
3,\nYou are a chatbot that helps with answering ...,Die Kriteriengewichtung für Teilnahmebezogene ...,1
4,\nYou are a chatbot that helps with answering ...,Die Kriteriengewichtung für Anbahnungsphase be...,1
5,\nYou are a chatbot that helps with answering ...,Die Kriteriengewichtung für Akutphase beträgt ...,1
6,\nYou are a chatbot that helps with answering ...,Die Kriteriengewichtung für Coachingphase betr...,1
7,\nYou are a chatbot that helps with answering ...,Die Kriteriengewichtung für Personal beträgt 1...,1
8,\nYou are a chatbot that helps with answering ...,Die Kriteriengewichtung für Personaleinsatz be...,1
9,\nYou are a chatbot that helps with answering ...,Die Kriteriengewichtung für Organisation beträ...,1


In [20]:
results.upload()


Evaluation results uploaded! View at https://app.ragas.io/dashboard/alignment/evaluation/99658072-6d53-4184-8f4c-492a1ab7e2d8


'https://app.ragas.io/dashboard/alignment/evaluation/99658072-6d53-4184-8f4c-492a1ab7e2d8'

Let's Try it out with our pdfRagChain Now.

First we need to modify our chain so that the filled out prompt template is returned by the chain invoke

In [21]:
from app.rag_chain import final_chain
load_dotenv()
# real_dataset.append(TestSample())
response = final_chain.invoke({"question":"Erkläre was die Firma EUBIA macht","collection_name":"test1"})
response["filled_prompt"]

ChatPromptValue(messages=[HumanMessage(content='\nYou are a chatbot that helps with answering questions. \nYou shall answer the questions only on the context provided.\nAvoid providing answers that are not relevant to the context.\nAvoid providing answers that are not relevant to the question.\nAnswer the question in the language of the question.\n\nAnswer given the following context:\n[Document(id=\'f5dae109-1b2c-44e0-a2ac-133f8a3cd16c\', metadata={\'source\': \'source_docs/EUBIA.txt\'}, page_content=\'Unternehmensdetails:\\n\\n* Name: EUBIA GmbH (Fort- und Weiterbildung)\\n* Schwerpunkt: Fort- und Weiterbildung\\n* Zielgruppe: Fachkräfte und Organisationen aus den Bereichen Sozialarbeit, Gesundheitswesen, Pädagogik und angrenzenden Feldern\\n* Standort: Bielefeld, Deutschland\\n* Adresse: EUBIA GmbH, Am Sparrenberg 8a, 33602 Bielefeld\\n* Telefon: 0521 - 55 75 36 0\\n* Fax: 0521 - 55 75 36 29\\n* E-Mail: [email address removed]\\n* Website: www.eubia.de\\n* Geschäftsführung: Dipl. Pä

Great! Now lets build a test dataset and evaluate our rag chain with RAGAS. Let's start simple and add the questions from the Coding Challenge

In [27]:
questions = ["Erkläre was die Firma EUBIA macht",
"Was ist für die Bewerbung erforderlich? Liste alle Dokumente, Anforderungen und Voraussetzungen.",
"Zitiere die genauen Anforderungen für die Bewerbung nach der Bewertungsmatrix?",
"Basierend auf den Anforderungen, erstelle eine Gliederung für die Bewerbung.",
]
real_dataset = []
for question in questions:
    result = final_chain.invoke({"question":question,"collection_name":"test1"})
    filled_prompt = result["filled_prompt"]
    answer = result["answer"]
    real_dataset.append(TestSample(user_input=filled_prompt.messages[0].content, response=answer.content).model_dump())

In [28]:
dataset = EvaluationDataset.from_list(real_dataset)
results = evaluate(dataset, metrics=[metric])
results.upload()

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluation results uploaded! View at https://app.ragas.io/dashboard/alignment/evaluation/0b7eb8fb-e4e8-461f-9206-a147f69b3a7c


'https://app.ragas.io/dashboard/alignment/evaluation/0b7eb8fb-e4e8-461f-9206-a147f69b3a7c'

In [ ]:
dataset.to_pandas()


,user_input,response
0,\nYou are a chatbot that helps with answering ...,Die Firma EUBIA GmbH ist ein deutscher Fortbil...
1,\nYou are a chatbot that helps with answering ...,Die Firma EUBIA GmbH ist ein deutscher Fortbil...
2,\nYou are a chatbot that helps with answering ...,Die Firma EUBIA GmbH ist ein deutscher Fortbil...
3,\nYou are a chatbot that helps with answering ...,Die Firma EUBIA GmbH ist ein deutscher Fortbil...
